In [ ]:
import json
from collections import Counter
from json import JSONDecodeError

CHUNK = 10 * 1024   # 10 KB


def process_log_lines(path: str) -> int:
    """
    Count how many lines in the file contain the substring 'ERROR'.
    Lines are separated by '\n'.
    """
    count = 0
    buffer = ""
    with open(path, "r", encoding="utf-8") as f:
        while chunk := f.read(CHUNK):
            buffer += chunk
            parts = buffer.split("\n")
            # all but last are full lines
            for line in parts[:-1]:
                if "ERROR" in line:
                    count += 1
            buffer = parts[-1]
    # handle final partial line
    if buffer and "ERROR" in buffer:
        count += 1
    return count


def process_ndjson(path: str, field: str = "value") -> int:
    """
    Sum the given field across every JSON object in an NDJSON file
    (one JSON object per line).
    """
    total = 0
    buffer = ""
    with open(path, "r", encoding="utf-8") as f:
        while chunk := f.read(CHUNK):
            buffer += chunk
            lines = buffer.split("\n")
            for raw in lines[:-1]:
                obj = json.loads(raw)
                total += obj.get(field, 0)
            buffer = lines[-1]
    # last line if it wasn’t newline-terminated
    if buffer.strip():
        obj = json.loads(buffer)
        total += obj.get(field, 0)
    return total


def process_concat_json(path: str) -> Counter:
    """
    Read a file of back-to-back JSON objects (no delimiters)
    and count occurrences of each obj['type'].
    """
    counts: Counter = Counter()
    buffer = ""
    decoder = json.JSONDecoder()

    with open(path, "r", encoding="utf-8") as f:
        while chunk := f.read(CHUNK):
            buffer += chunk
            while True:
                try:
                    obj, idx = decoder.raw_decode(buffer)
                except JSONDecodeError:
                    break
                counts[obj.get("type", "<missing>")] += 1
                buffer = buffer[idx:]

    # any trailing complete object?
    while buffer:
        try:
            obj, idx = decoder.raw_decode(buffer)
        except JSONDecodeError:
            break
        counts[obj.get("type", "<missing>")] += 1
        buffer = buffer[idx:]

    return counts
